In [15]:
import re
import string
import sqlite3
import io
import time
from datetime import timedelta
import gensim
from gensim.models import word2vec
import multiprocessing

In [2]:
#Menghubungkan ke database
from sqlite3 import Error
db_file = 'DBnews.db'
conn = None
try:
    conn = sqlite3.connect(db_file)
except Error as e:
    print(e)

In [4]:
#mengambil kolom comments untuk diolah
comments = []
cur = conn.cursor()
cur.execute("SELECT * from t_comments")
rows = cur.fetchall()
for row in rows:
    comments.append(row)

In [ ]:
comments

In [5]:
#Mengambil hanya konten comments
new_comments = []
for comment in comments:
    new_comments.append(comment[1])

In [ ]:
new_comments

In [6]:
#Preprocessing data (removePunctuation dan lowercase)

def removePunctuation(x):
    # Lowercasing all words
    x = x.lower()
    # Removing non ASCII chars
    x = re.sub(r'[^\x00-\x7f]',r' ',x)
    # Removing (replacing with empty spaces actually) all the punctuations
    return re.sub("["+string.punctuation+"]", " ", x)

pre_comments = []
for comment in new_comments:
    pre_comments.append(removePunctuation(comment))

In [53]:
pre_comments

['wkwkwk  gak sekalian pak  selawat ke dpr    kan banyak koruptor disana',
 'mantab   ini br dakwah yg sejati   kl d tempat umum mah dh biasa   ini luar biasa  ',
 'salut buat gus miftah',
 'yg nyinyir gak pernah lihat dan baca atw mengetahui sejarah dakwah walisongo \\  sunan kali jaga   smua ada cara demi kebaikan    asal jng ajk dg cara berjinah      lakukan dg porsi dan sikon yg tepat',
 'ada adabnya sholawat  lebih baik ajak ke majelis ilmu  ',
 'setiap ulama punya jalan dan cara dakwah masing2    jika kamu melihat kemunkaran lawan dengan tanganmu kekuatanmu  jika tidak mampu lawan dengan mulutmu jika tidak mampu juga dengan hatimu itulah selemah lemahnya iman    itu sudah menjadi kewajiban para ulama  kalo yg komen masih ada yg nyinyirin ulama berarti otaknya sudah disisipin politik  ',
 'umpanin nocannya yak   ',
 'itulah ceramah yg sebenarnya  ',
 'besok bugil aja selawat',
 'bedanya gus miftah dengan fpi    hti bagaikan surga dan neraka  di surga penuh kasih  di neraka penuh p

In [7]:
#Tokenisasi hasil preprocessing data
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('\s+', gaps=True)
tok_comments = []
for comment in pre_comments:
    tok_comments.append(tokenizer.tokenize(comment))

In [54]:
tok_comments

[['wkwkwk',
  'gak',
  'sekalian',
  'pak',
  'selawat',
  'ke',
  'dpr',
  'kan',
  'banyak',
  'koruptor',
  'disana'],
 ['mantab',
  'ini',
  'br',
  'dakwah',
  'yg',
  'sejati',
  'kl',
  'd',
  'tempat',
  'umum',
  'mah',
  'dh',
  'biasa',
  'ini',
  'luar',
  'biasa'],
 ['salut', 'buat', 'gus', 'miftah'],
 ['yg',
  'nyinyir',
  'gak',
  'pernah',
  'lihat',
  'dan',
  'baca',
  'atw',
  'mengetahui',
  'sejarah',
  'dakwah',
  'walisongo',
  '\\',
  'sunan',
  'kali',
  'jaga',
  'smua',
  'ada',
  'cara',
  'demi',
  'kebaikan',
  'asal',
  'jng',
  'ajk',
  'dg',
  'cara',
  'berjinah',
  'lakukan',
  'dg',
  'porsi',
  'dan',
  'sikon',
  'yg',
  'tepat'],
 ['ada',
  'adabnya',
  'sholawat',
  'lebih',
  'baik',
  'ajak',
  'ke',
  'majelis',
  'ilmu'],
 ['setiap',
  'ulama',
  'punya',
  'jalan',
  'dan',
  'cara',
  'dakwah',
  'masing2',
  'jika',
  'kamu',
  'melihat',
  'kemunkaran',
  'lawan',
  'dengan',
  'tanganmu',
  'kekuatanmu',
  'jika',
  'tidak',
  'mampu',
 

In [8]:
#Menghilangkan token yang kosong
for comment in tok_comments:
    for sub_comment in comment:
        if sub_comment == '':
            comment.remove(sub_comment)

In [ ]:
tok_comments

In [9]:
#Untuk mempercepat proses, gunakan 1000 data
tok_comments = tok_comments[:1000]

In [10]:
#Mengolah date menggunakan corpus yang sudah ada
start_time = time.time()
print('Streaming wiki...')
id_wiki = gensim.corpora.WikiCorpus('idwiki-latest-pages-articles.xml.bz2', lemmatize=False, dictionary={})
article_count = 0
with io.open('idwiki.txt', 'w', encoding='utf-8') as wiki_txt:
    for text in id_wiki.get_texts():
        wiki_txt.write(" ".join(text) + '\n')
        article_count += 1
        if article_count % 10000 == 0:
            print('{} articles processed'.format(article_count))
    print('total: {} articles'.format(article_count))
finish_time = time.time()
print('Elapsed time: {}'.format(timedelta(seconds=finish_time-start_time)))

Streaming wiki...


C:\Users\Mon\Anaconda3\lib\site-packages\gensim\utils.py:1268: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected %s; aliasing chunkize to chunkize_serial" % entity)


10000 articles processed
20000 articles processed
30000 articles processed
40000 articles processed
50000 articles processed
60000 articles processed
70000 articles processed
80000 articles processed
90000 articles processed
100000 articles processed
110000 articles processed
120000 articles processed
130000 articles processed
140000 articles processed
150000 articles processed
160000 articles processed
170000 articles processed
180000 articles processed
190000 articles processed
200000 articles processed
210000 articles processed
220000 articles processed
230000 articles processed
240000 articles processed
250000 articles processed
260000 articles processed
270000 articles processed
280000 articles processed
290000 articles processed
300000 articles processed
310000 articles processed
320000 articles processed
330000 articles processed
340000 articles processed
350000 articles processed
360000 articles processed
370000 articles processed
380000 articles processed
390000 articles proce

In [16]:
#pembuatan model word2vec
start_time = time.time()
print('Training Word2Vec Model...')
sentences = word2vec.LineSentence('idwiki.txt')
id_w2v = word2vec.Word2Vec(sentences, size=200, workers=multiprocessing.cpu_count()-1)
id_w2v.save('idwiki_word2vec.model')
finish_time = time.time()
print('Finished. Elapsed time: {}'.format(timedelta(seconds=finish_time-start_time)))

Training Word2Vec Model...
Finished. Elapsed time: 0:15:38.175323


In [55]:
#mencoba model yang dibuat
id_w2v.wv.most_similar("tak")

[('tidak', 0.8423470854759216),
 ('belum', 0.6460697054862976),
 ('tidaklah', 0.6128645539283752),
 ('tanpa', 0.605999231338501),
 ('takkan', 0.5660351514816284),
 ('tiada', 0.5582796931266785),
 ('jangan', 0.5192607045173645),
 ('tidah', 0.49242860078811646),
 ('saja', 0.4726674556732178),
 ('bukan', 0.47242629528045654)]

In [52]:
id_w2v.wv.most_similar("met")

[('een', 0.7381356358528137),
 ('ook', 0.7293915152549744),
 ('zijn', 0.7210648059844971),
 ('gegeven', 0.7163347601890564),
 ('twee', 0.7120093107223511),
 ('naar', 0.7032785415649414),
 ('mijn', 0.6957215070724487),
 ('niet', 0.6907348036766052),
 ('uit', 0.6903935670852661),
 ('wordt', 0.6892490386962891)]

In [18]:
#Memproses data slang/alay
import pandas as pd
slang_corpus = pd.read_csv("colloquial-indonesian-lexicon.csv")
slang_corpus = slang_corpus[slang_corpus.columns[0:2]]
slang_corpus

slang         formal
0            woww            wow
1           aminn           amin
2             met        selamat
3          netaas        menetas
4         keberpa       keberapa
5        eeeehhhh             eh
6      kata2nyaaa   kata-katanya
7           hallo           halo
8            kaka          kakak
9              ka            kak
10           daah            dah
11      aaaaahhhh             ah
12            yaa             ya
13           smga         semoga
14          slalu         selalu
15          amiin           amin
16             kk          kakak
17           trus          terus
18             kk          kakak
19            sii            sih
20      nyenengin   menyenangkan
21            bgt         banget
22         gemess          gemas
23          akuuu            aku
24            jgn         jangan
25            yaa             ya
26           udah          sudah
27           gitu         begitu
28            aja           saja
29       gemesiin   menggemaskan
...           ...            ...
14976          kl           kalo
14977        sich            sih
14978         pgn         pengin
14979     senang2  senang-senang
14980          jh           saja
14981         seh            sih
14982       nggak         enggak
14983  kereeennnn          keren
14984        donk           dong
14985         bln          bulan
14986         gmn      bagaimana
14987     gilakkk           gila
14988        gtu2    begitu-gitu
14989        sehh            sih
14990        kakk            kak
14991        ajah           saja
14992        cpet          cepat
14993       bnget         banget
14994          aq            aku
14995         uda          sudah
14996     nyobain        mencoba
14997         dlm          dalam
14998        wktu          waktu
14999          hr           hari
15000       gatau     enggak tau
15001    gataunya  enggak taunya
15002        gtau     enggak tau
15003       gatau     enggak tau
15004       fans2        fan-fan
15005     gaharus   enggak harus

[15006 rows x 2 columns]

In [19]:
#Menggunakan perbandingan dengan word2vec yang ada dan edit distance untuk membuat kata menjadi kata formal
print("Processing data...")
start_time = time.time()
import nltk
corr_comments = []
slang_words = []
reco_words = []
corr_word = 0
tot_word = 0
for comment in tok_comments:
    corr_comment = []
    slang_word = []
    for sub_comment in comment:
        if sub_comment in id_w2v.wv.vocab:
            sim_slang = slang_corpus[slang_corpus['slang'] == sub_comment]
            if len(sim_slang) != 0:
                mode_word = sim_slang.formal.mode()[0]
                corr_word = corr_word + 1
                corr_comment.append(mode_word)
            else:
                corr_word = corr_word + 1
                corr_comment.append(sub_comment)
        else:
            comment_found = False
            sim_slang = slang_corpus[slang_corpus['slang'] == sub_comment]
            if len(sim_slang) != 0:
                mode_word = sim_slang.formal.mode()[0]
                corr_word = corr_word + 1
                corr_comment.append(mode_word)
                comment_found = True
            else:
                rat_words = []
                for slang in slang_corpus['slang']:
                    rat_word = nltk.edit_distance(sub_comment, slang)
                    rat_words.append(rat_word)
                min_rat = rat_words.index(min(rat_words))
                value = str(slang_corpus.formal.loc[min_rat])
                corr_words = corr_word + 1
                corr_comment.append(value)
                slang_words.append(sub_comment + " = " + value)
                comment_found = True
            if comment_found ==  False:
                corr_comment.append(sub_comment)
                slang_words.append(sub_comment + " = " + "Not Found")
        tot_word = tot_word + 1
    corr_comments.append(corr_comment)
    print("Total kata yang sudah diproses : " + str(tot_word))
print("Total kata yang diperbaiki : " + str(corr_word) + " dari total semua kata : " + str(tot_word))
finish_time = time.time()
print('Elapsed time: {}'.format(timedelta(seconds=finish_time-start_time)))

Processing data...
Total kata yang sudah diproses : 11
Total kata yang sudah diproses : 27
Total kata yang sudah diproses : 31
Total kata yang sudah diproses : 65
Total kata yang sudah diproses : 74
Total kata yang sudah diproses : 125
Total kata yang sudah diproses : 128
Total kata yang sudah diproses : 132
Total kata yang sudah diproses : 136
Total kata yang sudah diproses : 154
Total kata yang sudah diproses : 168
Total kata yang sudah diproses : 182
Total kata yang sudah diproses : 213
Total kata yang sudah diproses : 234
Total kata yang sudah diproses : 240
Total kata yang sudah diproses : 251
Total kata yang sudah diproses : 260
Total kata yang sudah diproses : 265
Total kata yang sudah diproses : 270
Total kata yang sudah diproses : 276
Total kata yang sudah diproses : 284
Total kata yang sudah diproses : 296
Total kata yang sudah diproses : 309
Total kata yang sudah diproses : 322
Total kata yang sudah diproses : 338
Total kata yang sudah diproses : 342
Total kata yang sudah di

In [20]:
#komentar yang sudah diperbaiki
corr_comments

[['wkwkwk',
  'enggak',
  'sekalian',
  'pak',
  'selawat',
  'ke',
  'dpr',
  'kan',
  'banyak',
  'koruptor',
  'disana'],
 ['mantab',
  'ini',
  'baru',
  'dakwah',
  'yang',
  'sejati',
  'kalo',
  'di',
  'tempat',
  'umum',
  'mah',
  'dah',
  'biasa',
  'ini',
  'luar',
  'biasa'],
 ['salut', 'buat', 'gus', 'miftah'],
 ['yang',
  'nyinyir',
  'enggak',
  'pernah',
  'lihat',
  'dan',
  'baca',
  'atau',
  'mengetahui',
  'sejarah',
  'dakwah',
  'walisongo',
  'ada',
  'sunan',
  'kali',
  'jaga',
  'semua',
  'ada',
  'cara',
  'demi',
  'kebaikan',
  'asal',
  'jangan',
  'ajk',
  'dengan',
  'cara',
  'berzina',
  'lakukan',
  'dengan',
  'porsi',
  'dan',
  'sikon',
  'yang',
  'tepat'],
 ['ada',
  'adabnya',
  'sholawat',
  'lebih',
  'baik',
  'ajak',
  'ke',
  'majelis',
  'ilmu'],
 ['setiap',
  'ulama',
  'punya',
  'jalan',
  'dan',
  'cara',
  'dakwah',
  'masing-masing',
  'jika',
  'kamu',
  'melihat',
  'kemunkaran',
  'lawan',
  'dengan',
  'tanganmu',
  'kekuatanm

In [21]:
#Kata singkatan yang tidak ada pada slang corpus dan diolah dengan edit distance
slang_words

['\\ = ada',
 'disisipin = disini',
 'umpanin = menyarankan',
 'nocannya = menontonnya',
 'jadunya = jaman dulunya',
 'mantaaapppp = mantap',
 'josshhh = loh',
 'batas2 = bahas-bahas',
 'kereennnnnnn = keren',
 'snagat = ingat',
 'memviralkan = termiripkan',
 'boshe = boleh',
 'boshe = boleh',
 'ud83d = sudah',
 'ude02 = sudah-sudah',
 'ud83d = sudah',
 'ude02 = sudah-sudah',
 'begituan = begituan',
 'hahahaha = alah',
 'hastag = kasihan',
 'plesetkan = ledeki',
 'hahahaha = alah',
 'biaskan = biasa-biasa',
 'naaaah = deh',
 'bersholawat = selamat',
 'bersholawat = selamat',
 'bersholawat = selamat',
 'maaaaat = ya',
 'hahahaha = alah',
 'laah = dah',
 'ngikik = mengakak',
 'ngakui = mengaku',
 'nyapres = mencari',
 '2014 = tempat',
 '2019panbakalgurem = enggak bakal',
 'bakhir = menaksir',
 'sendiri2 = sendiri',
 'habis2an = habis-habis',
 'butek = bulat',
 'obok2 = sok-sok',
 'menseskab = membelah',
 '2 = ada',
 'mkan = bukan',
 'allahuakbar = allah',
 'hehehhe = sih',
 'grindong = g